<a href="https://colab.research.google.com/github/BumaranChe/Animal_Faces_Detection_Projects/blob/main/Option3_DNN_YOLOv8n_(GitHub).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.0 Download the class names file (coco.names)

In [ ]:
!wget -O coco.names https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

--2025-08-14 08:03:18--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2025-08-14 08:03:19 (37.7 MB/s) - ‘coco.names’ saved [625/625]



# 2.0 Loads yolov8n pretrained weights (PyTorch) and exports the model to ONNX format (making it ready to use with other runtimes OpenCV -->cv2.dnn)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.export(format="onnx", opset=12, dynamic=False, simplify=True, nms=False)

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.64...
ONNX: export success ✅ 1.3s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (2.3s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8n.onnx'

# 3.0 Load YOLO model into OpenCV's DNN module

In [ ]:
import cv2
import numpy as np
import os

# ===============================
# CONFIG
# ===============================
ONNX_MODEL = "yolov8n.onnx"  # Exported with nms=True
COCO_NAMES = "coco.names"
ANIMAL_CLASSES = ['cat', 'dog', 'cow', 'horse', 'sheep', 'bird']
CONF_THRESHOLD = 0.5

# ===============================
# Load COCO class labels
# ===============================
with open(COCO_NAMES, "r") as f:
    classes = [line.strip() for line in f]

# ===============================
# Prepare folders
# ===============================
os.makedirs("positives", exist_ok=True)
os.makedirs("negatives", exist_ok=True)

# ===============================
# Load ONNX YOLO model
# ===============================
net = cv2.dnn.readNetFromONNX(ONNX_MODEL)

# ===============================
# Process all JPG images in folder
# ===============================
for img_name in os.listdir():
    if not img_name.lower().endswith(".jpg"):
        continue

    img = cv2.imread(img_name)
    if img is None:
        continue

    height, width = img.shape[:2]

    # Create input blob
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (640, 640), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward()

    # YOLOv8 NMS output format: [batch, num_detections, 6]
    # Each detection: [x1, y1, x2, y2, score, class_id]
    detections = detections[0]  # Remove batch dim

    animal_found = False

    for det in detections:
        score = det[4]
        class_id = int(det[5])

        if score < CONF_THRESHOLD:
            continue

        if class_id < len(classes) and classes[class_id] in ANIMAL_CLASSES:
            animal_found = True

            x1, y1, x2, y2 = map(int, det[:4])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"{classes[class_id]} {score:.2f}",
                        (x1, max(15, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

    # Save to correct folder
    if animal_found:
        cv2.imwrite(f"positives/{img_name}", img)
    else:
        cv2.imwrite(f"negatives/{img_name}", img)

print("✅ Processing completed.")

✅ Processing completed.


After loading images in content/ , the image file will be processed and saved into content/positives or content/negatives folder. Based on the testing, this model detection is not working well because some of the background (negative images) save into positive directory (content/positives). In conclusion, this model doesn't work well